In [19]:
import pandas

In [20]:
data = pandas.read_csv('data.csv')

In [21]:
def get_interpolated(values, times, start, end, step, delta):
    current = []
    current_position = 0
    time_now = start
    result_values = []
    result_times = []
    while time_now <= end:
        while (current_position + 1 < times.shape[0]) and (times[current_position + 1] <= time_now):
            current_position = current_position + 1
            
        if current_position  + 1 == times.shape[0]:
            result_times.append(int(time_now))
            result_values.append('')
        else:
            delta_left = abs(times[current_position] - time_now)
            delta_right = abs(times[current_position + 1] - time_now)
            if delta_left > delta or delta_right > delta:
                result_times.append(int(time_now))
                result_values.append('')
            else:
                value_left = values[current_position]
                value_right = values[current_position + 1]
                value_interpolated = (value_left * delta_right + value_right * delta_left) / \
                                    (delta_left + delta_right)
                result_times.append(int(time_now))
                result_values.append(value_interpolated)
        time_now += step
    return result_times, result_values

In [22]:
def is_correct_market(name, market):
    splitted = name.split('_')
    if len(splitted) != 2:
        return False
    return splitted[0][-3:] == market
    

In [23]:
def extract_currencies(columns):
    result = []
    for column in columns:
        first = column.split('_')[0]
        currency_now = first[0:len(first) - 3]
        result.append(currency_now)
    return list(set(result))

In [24]:
import numpy as np
import tqdm
def extract_data(data, step, delta, market):
    correct_columns = [column for column in data.columns if is_correct_market(column, market)]
    currencies  = extract_currencies(correct_columns)
    
    mins, maxs = [], []
    for currency in currencies:
        timestamps_now = data[currency + market + '_timestamp'].as_matrix()
        min_now = np.min(timestamps_now)
        max_now = np.max(timestamps_now)
        mins.append(min_now)
        maxs.append(max_now)

    min_time = np.min(mins)
    max_time = np.max(maxs)
    
    result = {}
    for currency in tqdm.tqdm(currencies):
        asks_now = data[currency + market + '_ask'].as_matrix()
        bids_now = data[currency + market + '_bid'].as_matrix()
        timestamps_now = data[currency + market + '_timestamp'].as_matrix()
        asks_prepared_now = get_interpolated(asks_now, timestamps_now, min_time, max_time, step, delta)
        bids_prepared_now = get_interpolated(bids_now, timestamps_now, min_time, max_time, step, delta)
        
        times = asks_prepared_now[0]
        asks = asks_prepared_now[1]
        bids = bids_prepared_now[1]
        
        result['timestamps'] = np.asarray(times) / 1000.0
        result[currency + '_asks'] = asks
        result[currency + '_bids'] = bids
    result = pandas.DataFrame(result)
    columns_except_timestamps = [column for column in result.columns if column != 'timestamps']
    result = result[['timestamps'] + columns_except_timestamps]
    return result
        

In [28]:
def extract_all(data):
    extracts = []
    result = None
    for market in ['BTC', 'USD', 'ETH']:
        data_extracted = extract_data(data, 100.0 * 1000, 5.0 * 1000, market)
        data_extracted.to_csv("data_extracted_" + market + '.csv', sep = ';', index = False)
        #print(data_extracted.shape)
        #print(data_extracted.head())
        if result is None:
            result = data_extracted
            renaming = {}
            for column in result.columns:
                if column != 'timestamps':
                    renaming[column] = column + '__' + market
            #print(renaming)
            result = result.rename(columns = renaming)
            #print(result.columns)
        else:
            renaming = {}
            for column in data_extracted.columns:
                if column != 'timestamps':
                    renaming[column] = column + '__' + market
            data_extracted = data_extracted.rename(columns = renaming)
            result = result.join(data_extracted.set_index('timestamps'), on = 'timestamps')
            #print(result.columns)
    #print(result[0].columns)
    #result = pandas.concat(result, axis = 'timestamps')
    #print(result.columns)
    result.to_csv("data_extracted_joint.csv", sep = ';', index = False)
    #print(result.head())
    #print(result.shape)
    

In [29]:
extract_all(data)

100%|██████████| 6/6 [01:45<00:00, 17.47s/it]
